# Νευρωνικά Δίκτυα και Κείμενο (Ελάχιστη Εισαγωγή)

Υλικό προσαρμοσμένο από [σχετικό παράδειγμα της τεκμηρίωσης του TensorFlow](https://www.tensorflow.org/tutorials/keras/text_classification).

---

> Πάνος Λουρίδας, Αναπληρωτής Καθηγητής <br />
> Τμήμα Διοικητικής Επιστήμης και Τεχνολογίας <br />
> Οικονομικό Πανεπιστήμιο Αθηνών <br />
> louridas@aueb.gr

In [1]:
import tensorflow as tf

from tensorflow import keras
from keras import layers, losses
import matplotlib.pyplot as plt

import os
import re
import string

* Θα χρησιμοποιήσουμε ένα σύνολο δεδομένων το οποίο περιέχει το κείμενο 50.000 κριτικών από το [Internet Movie Database (IMDb)](https://www.imdb.com/).

* Αυτές είναι διαχωρισμένες σε 25.000 κριτικές για εκπαίδευση και 25.000 κριτικές για έλεγχο.

* Σκοπός μας είναι να φτιάξουμε ένα μοντέλο το οποίο θα διαβάζει μία κριτική και θα μπορεί να αποφανθεί αν είναι θετική ή αρνητική.

* Για περισσότερες πληροφορίες βλ. Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. 2011. Learning Word Vectors for Sentiment Analysis. In Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies, pages 142–150, Portland, Oregon, USA. Association for Computational Linguistics. Διαθέσιμο στο <https://aclanthology.org/P11-1015>.

* Θα πρέπει να αποσυμπιέσουμε το αρχείο `aclImdb.zip`.

* Θα δημιουργηθεί ένας κατάλογος `aclImdb`.

In [2]:
dataset_dir = 'aclImdb'

* Τα δεδομένα περιέχονται σε δύο υποκαταλόγους, `train` και `test`.

In [3]:
os.listdir(dataset_dir)

['README', 'test', 'train']

* Για να δούμε τι υπάρχει στον κατάλογο `train`.


In [4]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['neg', 'pos']

* O κατάλογος `pos` περιέχει θετικές κριτικές, κάθε μία σε ένα ξεχωριστό αρχείο.

* Ομοίως, ο κατάλογος `neg` περιέχει αρνητικές κριτικές, κάθε μία σε ένα ξεχωριστό αρχείο.

* Για να δούμε μία κριτική.

In [5]:
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
    print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


* Για να πάρουμε τα δεδομένα μέσα στο TensorFlow θα χρησιμοποιήσουμε τη συνάρτηση `text_dataset_from_directory()`.

* Αυτή προϋποθέτει ότι τα δεδομένα μας είναι τοποθετημένα σε καταλόγους, έναν για κάθε διακριτή κλάση, δηλαδή:

  ```
  main_directory/
  ...class_a/
  ......a_text_1.txt
  ......a_text_2.txt
  ...class_b/
  ......b_text_1.txt
  ......b_text_2.txt
  ```
  
* Αυτό ακριβώς έχουμε ήδη.

* Θα πάρουμε το υποσύνολο εκπαίδευσης.

* Θα κρατήσουμε 20% για επικύρωση (πέρα από τα δεδομένα ελέγχου).

In [6]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


* Ας δούμε μερικές κριτικές και την κλάση στην οποία ανήκουν.

In [7]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(0, 2+1):
        print("Review", text_batch.numpy()[i])
        print("Label", label_batch.numpy()[i])

Review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into 

* Για να δούμε σε τι αντιστοιχούν οι δύο κλάσεις `0` και `1` μπορούμε να χρησιμοποιήσουμε την ιδιότητα  `class_names`.

In [8]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to neg
Label 1 corresponds to pos


* Αφού πήραμε το υποσύνολο εκπαίδευσης, θα πάρουμε τώρα το υποσύνολο επικύρωσης. 

* Προσέξτε ότι χρησιμοποιούμε τον ίδιο σπόρο (`seed`) για να είμαστε σίγουροι ότι δεν θα υπάρχουν επικαλύψεις μεταξύ των δεδομένων εκπαίδευσης και επικύρωσης.

In [9]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


* Τέλος, παίρνουμε και τα δεδομένα ελέγχου.

In [10]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


* Όπως είδαμε προηγουμένως, οι κριτικές περιέχουν εκτός από κείμενο και την αλλαγή γραμμής σε HTML (`<br />`). 

* Εμείς θα τα αφαιρέσουμε αυτά.

* Επίσης θα κάνουμε όλους τους χαρακτήρες πεζούς και θα αφαιρέσουμε σημεία στίξης.

In [11]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                    '[%s]' % re.escape(string.punctuation),
                                    '')

* Στη συνέχεια θα φτιάξουμε ένα στρώμα `TextVectorization`.

* Αυτό το στρώμα θα κάνει τις παρακάτω προεργασίες:

  * Θα καλέσει την `custom_standardization()`.

  * Θα χωρίσει κάθε συμβολοσειρά που αντιστοιχεί σε μία κριτική σε επιμέρους λεκτικές μονάδες (tokens), χρησιμοποιώντας ως διαχωριστή χαρακτήρες κενών.
  
  * Θα αντιστοιχίσει σε κάθε μία από τις πιο συχνά εμφανιζόμενες λεκτικές μονάδες έναν ακέραιο αριθμό, δημιουργώντας ένα λεξικό μεγέθους 10.000.
  
  * Θα εξασφαλίσει ότι κάθε σειρά ακεραίων που θα προκύψει (που θα αναπαριστά κάθε κριτική) θα έχει το ίδιο μήκος.

In [12]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

* Καλούμε τη μέθοδο `adapt()` του `vectorize_layer` στο σύνολο εκπαίδευσης ώστε να κατασκευαστεί η αντιστοίχιση (λεξικό, vocabulary) μεταξύ μονάδων και ακεραίων. 

In [13]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

* Για να δούμε καλύτερα τι κάνει το `vectorize_layer`, θα γράψουμε μια βοηθητική συνάρτηση με την οποία θα μπορούμε να το καλούμε στα δεδομένα μας.

In [14]:
def vectorize_text(text, label):
    # text is a tensor with shape (), we need to make it with shape (1,)
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [15]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />T

* Όπως βλέπουμε, κάθε λεκτική μονάδα αναπαρίσταται από έναν ακέραιο αριθμό.

* Ο αριθμός 0 χρησιμοποιείται για γέμισμα (padding), ώστε κάθε κριτική να έχει το ίδιο μήκος.

* Ο αριθμός 1 χρησιμοποιείται για άγνωστες λέξεις, δηλαδή λέξεις που είναι εκτός των 10.000 συχνότερων, τις οποίες κρατάει το λεξικό μας.

In [16]:
print("0 ---> ", vectorize_layer.get_vocabulary()[0])
print("1 ---> ",vectorize_layer.get_vocabulary()[1])
print("1287 ---> ", vectorize_layer.get_vocabulary()[1287])
print("9999 ---> ",vectorize_layer.get_vocabulary()[9999])
print("Vocabulary size: ", vectorize_layer.vocabulary_size())

0 --->  
1 --->  [UNK]
1287 --->  silent
9999 --->  rushes
Vocabulary size:  10000


* Για να βελτιώσουμε την ταχύτητα, θα χρησιμοποιήσουμε τις μεθόδους `cache()` και `prefetch()`. 

* Με τη μέθοδο `cache()`, τα δεδομένα την πρώτη φορά που διαβάζονται μπορούν να αποθηκευτούν στη μνήμη.

* Με τη μέθοδο `prefetch()`, τα δεδομένα τροφοδούνται στο νευρωνικό δίκτυο καθώς το δίκτυο επεξεργάζεται ήδη τα προηγούμενα δεδομένα, ώστε να μην χάνεται χρόνος για την τροφοδοσία (αφού μπορεί να γίνεται παράλληλα με την επεξεργασία).

In [17]:
AUTOTUNE = tf.data.AUTOTUNE

raw_train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
raw_val_ds = raw_val_ds.cache().prefetch(buffer_size=AUTOTUNE)
raw_test_ds = raw_test_ds.cache().prefetch(buffer_size=AUTOTUNE)

* Προχωράμε στην κατασκευή του μοντέλου μας.

* Το πρώτο επίπεδο που βάζουμε στο μοντέλο μας είναι το `vectorize_layer`.

In [18]:
model = tf.keras.Sequential()

model.add(vectorize_layer)

* Στη συνέχεια θα προσθέσουμε ένα *στρώμα ενσωμάτωσης* (embedding layer).

* Το στρώμα αυτό μετατρέπει τον ακέραιο αριθμό που αναπαριστά κάθε λεκτική μονάδα σε ένα *διάνυσμα* 16 διαστάσεων (δική μας επιλογή).

* Επομένως περνάμε από μια αναπαράσταση «λέξη-αριθμός» σε μία αναπαράσταση «λέξη-διάνυσμα». 

* Η διανυσματική αυτή αναπαράσταση, η ενσωμάτωση, θα εκφράζει κατά κάποιον τρόπο το νόημα της κάθε λεκτικής μονάδας.

* Πώς προκύπτει η διανυσματική αναπαράσταση κάθε λέξης; Θα τη μάθει το δίκτυο!

* Η είσοδος του στρώματος ενσωμάτωσης είναι μια ακολουθία ακεραίων αριθμών, μήκους 250.

* Η έξοδος του στρώματος θα είναι πλέον ένας πίνακας διαστάσεων $250 \times 16$.

In [19]:
embedding_dim = 16

model.add(layers.Embedding(max_features, embedding_dim))

* Aκολουθεί ένα στρώμα dropout.

In [20]:
model.add(layers.Dropout(0.2))

* Κάθε κριτική αναπαραστάται με έναν πίνακα διαστάσεων $250 \times 16$.

* Από αυτήν θα παράξουμε ένα διάνυσμα με 16 διαστάσεις.

* Αυτό θα το κάνουμε με ένα στρώμα `GlobalAveragePooling1D`.

* Από τα 250 διανύσματα 16 διαστάσεων θα πάρουμε το μέσο όρο τους.

* Διαισθητικά, αυτή θα είναι η διανυσματική αναπαράσταση του «μέσου όρου» του νοήματος των λεκτικών μονάδων της κάθε κριτικής.

* Ακόμα πιο διαισθητικά, αυτή θα αντιστοιχεί το νόημα (σε μία ιδεατή λέξη) που συνοψίζει όλη την κριτική.

In [21]:
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dropout(0.2))

* Τέλος, θα προσθέσουμε ένα πυκνά συνδεμένο νευρώνα στο τελευταίο στρώμα ώστε να κάνουμε την ταξινόμηση.

In [22]:
model.add(layers.Dense(1))

* Να δούμε συνοπτικά τι έχουμε:

In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (1, 250)               │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

* Όπως συνήθως, ορίζουμε τον βελτιστοποιητή, απώλεια, και μετρική. 

In [24]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0)])

* Προχωράμε στην εκπαίδευση για δέκα εποχές.

In [25]:
epochs = 10
history = model.fit(
    raw_train_ds,
    validation_data=raw_val_ds,
    epochs=epochs)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - binary_accuracy: 0.5892 - loss: 0.6782 - val_binary_accuracy: 0.7390 - val_loss: 0.6042
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - binary_accuracy: 0.7629 - loss: 0.5712 - val_binary_accuracy: 0.8100 - val_loss: 0.4941
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - binary_accuracy: 0.8275 - loss: 0.4600 - val_binary_accuracy: 0.8276 - val_loss: 0.4267
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - binary_accuracy: 0.8547 - loss: 0.3894 - val_binary_accuracy: 0.8462 - val_loss: 0.3823
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - binary_accuracy: 0.8712 - loss: 0.3440 - val_binary_accuracy: 0.8488 - val_loss: 0.3596
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - binary_accuracy: 0.8827 - loss: 0.3114 - val_binary_accuracy: 0.8556 - val_loss: 0.3412
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - binary_accuracy: 0.8929 - loss: 0.2882 - val_binary_accuracy: 0.8562 - val_loss: 0.3311
Epoc

* Αφού εκπαιδεύσουμε, μπορούμε να δούμε την επίδοση στα δεδομένα ελέγχου.


In [26]:
loss, accuracy = model.evaluate(raw_test_ds)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - binary_accuracy: 0.8546 - loss: 0.3328
Loss:  0.335397332906723
Accuracy:  0.8518000245094299


* Ας επανέλθουμε λίγο στη διανυσματική αναπαράσταση των λέξεων στο μοντέλο μας.

* Τα διανύσματα για την κάθε λέξη είναι τα βάρη του στρώματος `Embedding`.

* Το στρώμα αυτό έχει διαστάσεις `(vocabulary_size, embedding_dim)`.

* Το νευρωνικό δίκτυο έμαθε τα βάρη, δηλαδή έμαθε τις διανυσματικές αναπαραστάσεις των λέξεων κατά την εκπαίδευση για την ταξινόμηση.

In [27]:
embedding = model.layers[1]
weights = embedding.get_weights()[0]
weights.shape

(10000, 16)

* Επομένως από εκεί που είχαμε ξεκινήσει με την αναπαράσταση των λέξεων μέσω ακεραίων, τελικά αναπαριστούμε κάθε λέξη ως ένα σημείο σε έναν χώρο 16 διαστάσεων.

In [28]:
for num in range(0, 5+1):
    word = vectorize_layer.get_vocabulary()[num]
    vec = weights[num]
    print(word, num, vec)

 0 [-0.0728702  -0.01296999  0.01166384 -0.00197988 -0.00370922  0.01953229
 -0.01916091  0.04223778  0.05442509 -0.04767247 -0.05128874 -0.02498573
 -0.02463265  0.04177372  0.01177678  0.02349255]
[UNK] 1 [ 0.0182911  -0.03255159 -0.03636377 -0.05057201 -0.00981174 -0.00292191
 -0.04862974 -0.03164761  0.1196816   0.01412571 -0.14138067  0.00033114
  0.05463718  0.09093923  0.03010117 -0.0052721 ]
the 2 [ 0.13252784  0.09988928 -0.08586258  0.11617798 -0.03390299 -0.15994595
  0.0639359   0.10116175 -0.001217    0.01978775  0.10478612  0.06842244
  0.00038935  0.00095713 -0.01489006 -0.10424837]
and 3 [ 0.10833963  0.17168185 -0.25671208  0.18921627 -0.29804635 -0.10761049
  0.13862373  0.19230205 -0.12879223  0.24387413  0.16962902  0.08816475
 -0.15664962 -0.17222846 -0.14667194 -0.21387216]
a 4 [ 0.08616402  0.16352668 -0.07304269 -0.01422404 -0.12385229 -0.18006712
  0.05604468  0.04547384 -0.14111122  0.06913619  0.02856174  0.0764794
 -0.07205418 -0.01644458  0.07001681 -0.0807

* Η χρήση διανυσματικών αναπαραστάσεων λέξεων είναι η βάση στα νευρωνικά δίκτυα που χειρίζονται γλώσσα.

* Εμείς στο απλό παράδειγμά μας χρησιμοποιήσαμε ένα μικρό σύνολο δεδομένων για την εκμάθηση των διανυσμάτων.

* Στην πράξη, υπάρχουν διαθέσιμες διανυσματικές αναπαραστάσεις λέξεων που έχουν προκύψει από εκπαίδευση σε τεράστια σώματα κειμένου.